In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn as sk
import scipy.stats as sc
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from ISLP import load_data

In [9]:
Carseats = load_data('Carseats')
print(Carseats.columns)
print(Carseats.head(8))

Index(['Sales', 'CompPrice', 'Income', 'Advertising', 'Population', 'Price',
       'ShelveLoc', 'Age', 'Education', 'Urban', 'US'],
      dtype='object')
   Sales  CompPrice  Income  Advertising  Population  Price ShelveLoc  Age  \
0   9.50        138      73           11         276    120       Bad   42   
1  11.22        111      48           16         260     83      Good   65   
2  10.06        113      35           10         269     80    Medium   59   
3   7.40        117     100            4         466     97    Medium   55   
4   4.15        141      64            3         340    128       Bad   38   
5  10.81        124     113           13         501     72       Bad   78   
6   6.63        115     105            0          45    108    Medium   71   
7  11.85        136      81           15         425    120      Good   67   

   Education Urban   US  
0         17   Yes  Yes  
1         10   Yes  Yes  
2         12   Yes  Yes  
3         14   Yes  Yes  
4         13

In [11]:
Carseats['US'] = Carseats['US'].replace({"Yes" : 1, "No" : 0})
Carseats['Urban'] = Carseats['Urban'].replace({"Yes" : 1, "No" : 0})
print(Carseats.head(8))

   Sales  CompPrice  Income  Advertising  Population  Price ShelveLoc  Age  \
0   9.50        138      73           11         276    120       Bad   42   
1  11.22        111      48           16         260     83      Good   65   
2  10.06        113      35           10         269     80    Medium   59   
3   7.40        117     100            4         466     97    Medium   55   
4   4.15        141      64            3         340    128       Bad   38   
5  10.81        124     113           13         501     72       Bad   78   
6   6.63        115     105            0          45    108    Medium   71   
7  11.85        136      81           15         425    120      Good   67   

   Education Urban US  
0         17     1  1  
1         10     1  1  
2         12     1  1  
3         14     1  1  
4         13     1  0  
5         16     0  1  
6         15     1  0  
7         10     1  1  


In [35]:
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(Carseats[['Price', 'Urban', 'US']], Carseats['Sales'], test_size=0.3)
formula = "Sales ~ " + " + ".join(X_train.columns)
df_train = pd.concat([X_train, y_train], axis = 1)

print(X_train.shape)
for cols in X_train:
    print(cols, X_train[cols].isna().count())

print(df_train[0:3])
model = ols(formula, data=df_train).fit()

print(model.params)

(280, 3)
Price 280
Urban 280
US 280
     Price Urban US  Sales
239    118     1  1   3.89
206    160     1  1   4.97
386    160     1  0   5.32
Intercept     13.900070
Urban[T.1]    -0.062616
US[T.1]        1.207737
Price         -0.062236
dtype: float64


In [37]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Sales   R-squared:                       0.287
Model:                            OLS   Adj. R-squared:                  0.279
Method:                 Least Squares   F-statistic:                     37.06
Date:                Fri, 22 Aug 2025   Prob (F-statistic):           3.73e-20
Time:                        11:34:59   Log-Likelihood:                -643.63
No. Observations:                 280   AIC:                             1295.
Df Residuals:                     276   BIC:                             1310.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     13.9001      0.775     17.927      0.0

In [39]:
new_cols = ['US', 'Price'] #P-value < 0.05
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(Carseats[new_cols], Carseats['Sales'], test_size=0.3)
formula = "Sales ~ " + " + ".join(X_train.columns)
df_train = pd.concat([X_train, y_train], axis = 1)
model = ols(formula, data=df_train).fit()
print(model.summary())
print(model.conf_int(alpha=0.05))

                            OLS Regression Results                            
Dep. Variable:                  Sales   R-squared:                       0.234
Model:                            OLS   Adj. R-squared:                  0.229
Method:                 Least Squares   F-statistic:                     42.41
Date:                Fri, 22 Aug 2025   Prob (F-statistic):           8.54e-17
Time:                        11:41:26   Log-Likelihood:                -653.67
No. Observations:                 280   AIC:                             1313.
Df Residuals:                     277   BIC:                             1324.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     13.3004      0.792     16.793      0.0

In [53]:
influence = model.get_influence()
resid_student = influence.resid_studentized_internal

resid_student = resid_student.reshape(-1, 1)
resid_student = np.array(resid_student)
print( len(resid_student[(resid_student > 3)]) )

0
